In [1]:
import pandas as pd
import json
import requests
from tqdm.notebook import tqdm
from datetime import datetime, timedelta

season = '22-23'

In [2]:
master_data = []

with open(f'data/{season}/waybacks.json', 'r') as json_file:
    snapshot_dates = json.load(json_file)
json_file.close()

for date in snapshot_dates:
    with open(f'data/{season}/bootstrap_statics/dump/{date}.json', 'r', encoding='UTF-8') as json_file:
        date_file = json.load(json_file)
    json_file.close()

    gameweek = [round['id'] for round in date_file['events'] if round['is_current'] == True]

    if gameweek == []:
        gameweek = 1
    else:
        gameweek = gameweek[0]

    finished = [round for round in date_file['events'] if (round['id'] == gameweek) and (round['finished'] == True)]

    if finished == []:
        finished = False
    else:
        finished = True


    extracted_data = date_file['elements']
    for element in extracted_data:
        element['gameweek'] = gameweek
        element['finished'] = finished
        element['timestamp'] = date
        
    master_data += extracted_data

In [5]:
data = pd.DataFrame(master_data)

data['timestamp'] = pd.to_datetime(data['timestamp'])

In [6]:
data.sort_values(['id', 'timestamp'])[['id', 'timestamp', 'ep_this', 'ep_next', 'gameweek', 'finished']].head(40)

id           timestamp ep_this ep_next  gameweek  finished
0       1 2022-08-02 20:51:54    None     2.3         1     False
536     1 2022-08-03 12:51:07    None     2.3         1     False
1094    1 2022-08-03 22:31:01    None     2.3         1     False
1652    1 2022-08-04 18:07:09    None     2.3         1     False
2210    1 2022-08-05 09:08:43    None     2.3         1     False
2768    1 2022-08-05 17:16:51    None     2.3         1     False
3327    1 2022-08-06 08:27:59     2.3     2.3         1     False
3886    1 2022-08-06 19:57:55     2.4     2.4         1     False
4454    1 2022-08-07 16:57:24     2.4     2.4         1     False
5022    1 2022-08-08 04:56:17     2.4     2.4         1      True
5595    1 2022-08-11 20:52:44     2.4     2.4         1      True
6175    1 2022-08-13 11:18:38     0.5     1.0         2     False
6755    1 2022-08-13 21:45:15     0.5     1.0         2     False
7335    1 2022-08-14 22:44:30     0.5     1.0         2     False
7915    1 2022-08-15 07:01:10     0.5     1.0         2     False
8495    1 2022-08-15 23:01:45     0.5     1.0         2     False
9079    1 2022-08-16 09:01:26     0.5     1.0         2      True
9664    1 2022-08-17 08:52:36     0.5     1.0         2      True
10249   1 2022-08-17 20:47:45     0.5     1.0         2      True
10836   1 2022-08-19 21:58:24     0.5     1.0         2      True
11425   1 2022-08-20 08:46:47     0.5     1.0         2      True
12014   1 2022-08-20 21:48:05     1.0     1.0         3     False
12605   1 2022-08-21 10:03:40     1.0     1.0         3     False
13196   1 2022-08-21 21:48:01     1.0     1.0         3     False
13787   1 2022-08-22 09:01:43     1.0     1.0         3     False
14378   1 2022-08-22 20:47:14     1.0     1.0         3     False
14969   1 2022-08-23 10:01:47     1.0     1.0         3      True
15561   1 2022-08-23 22:35:36     1.0     1.0         3      True
16154   1 2022-08-24 22:39:12     1.0     1.0         3      True
16747   1 2022-08-25 15:04:08     1.0     1.0         3      True
17340   1 2022-08-27 08:48:20     1.0     1.0         3      True
17934   1 2022-08-27 22:31:56     1.0     0.5         4     False
18535   1 2022-08-28 21:56:39     1.0     0.5         4     False
19136   1 2022-08-29 21:30:00     1.0     0.5         4      True
19737   1 2022-08-31 04:51:38     0.5     0.0         5     False
20343   1 2022-08-31 15:41:22     0.5     0.0         5     False
20949   1 2022-09-01 14:55:39     0.5     0.0         5     False
21559   1 2022-09-02 13:11:25     0.5     0.0         5      True
22182   1 2022-09-02 22:04:35     0.5     0.0         5      True
22805   1 2022-09-03 09:32:15     0.5     0.0         5      True

In [97]:
date_1 = dates[9]
date_2 = dates[10]

with open(f'data/{season}/bootstrap_statics/dump/{date_1}.json', 'r') as json_file:
    data_1 = json.load(json_file)
json_file.close()

with open(f'data/{season}/bootstrap_statics/dump/{date_2}.json', 'r') as json_file:
    data_2 = json.load(json_file)
json_file.close()

data_1['elements'] = {element['id']: element for element in data_1['elements']}
data_2['elements'] = {element['id']: element for element in data_2['elements']}


attribute = 'elements'

this_event_1 = [date_1] + [{'round': event['name'],'finished': event['finished']} for event in data_1['events'] if event['is_current'] or (event['id'] == 1)]
this_event_2 = [date_2] + [{'round': event['name'],'finished': event['finished']} for event in data_2['events'] if event['is_current'] or (event['id'] == 1)]

print(this_event_1)
print(this_event_2)

for n in data_1[attribute]:

    if data_2[attribute].get(n):

        diff_dict_1 = [{key: data_1[attribute][n][key]} for key in data_1[attribute][n] if data_1[attribute][n][key] != data_2[attribute][n][key]]
        diff_dict_2 = [{key: data_2[attribute][n][key]} for key in data_1[attribute][n] if data_1[attribute][n][key] != data_2[attribute][n][key]]

        playing_chance_1 = data_1[attribute][n]['chance_of_playing_next_round']
        if playing_chance_1 == None:
            playing_chance_1 = 1
        else:
            playing_chance_1 = float(playing_chance_1)/100

        playing_chance_2 = data_2[attribute][n]['chance_of_playing_next_round']
        if playing_chance_2 == None:
            playing_chance_2 = 1
        else:
            playing_chance_2 = float(playing_chance_2)/100

        if playing_chance_1 == 0:
            ep_next_true_1 = 0
        else:
            ep_next_true_1 = round(float(data_1[attribute][n]['ep_next'])/playing_chance_1, 1)

        if playing_chance_2 == 0:
            ep_next_true_2 = 0
        else:
            ep_next_true_2 = round(float(data_2[attribute][n]['ep_next'])/playing_chance_2, 1)

        if abs(ep_next_true_1 - ep_next_true_2) > 0.101 and (playing_chance_2 > 0) and (playing_chance_1 > 0):
            print('-'*100)
            print('ep_next_true', ep_next_true_1, diff_dict_1)
            print('ep_next_true', ep_next_true_2, diff_dict_2)
    else:
        print(n, 'missing')

['20220811', {'round': 'Gameweek 1', 'finished': True}]
['20220812', {'round': 'Gameweek 1', 'finished': True}, {'round': 'Gameweek 2', 'finished': False}]
----------------------------------------------------------------------------------------------------
ep_next_true 2.4 [{'cost_change_start': 0}, {'cost_change_start_fall': 0}, {'ep_next': '2.4'}, {'ep_this': '2.4'}, {'now_cost': 45}, {'transfers_in': 1087}, {'transfers_in_event': 1087}, {'transfers_out': 5522}, {'transfers_out_event': 5522}, {'influence_rank': 344}, {'influence_rank_type': 147}, {'creativity_rank': 340}, {'creativity_rank_type': 144}, {'threat_rank': 278}, {'threat_rank_type': 112}, {'ict_index_rank': 360}, {'ict_index_rank_type': 147}]
ep_next_true 1.0 [{'cost_change_start': -1}, {'cost_change_start_fall': 1}, {'ep_next': '1.0'}, {'ep_this': '0.5'}, {'now_cost': 44}, {'transfers_in': 1367}, {'transfers_in_event': 6}, {'transfers_out': 6540}, {'transfers_out_event': 10}, {'influence_rank': 334}, {'influence_rank_typ